In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D

In [97]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :4]

        past_candles = []
        #for z in range(1, how_many_past_candles+1):
        #  past_candles.append(data[i-z][3])
        #past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append(data[i+z][3])
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > data[i-1][3]:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close"])
        del rows
        del past_candles
        del next_candles
        dfs.append(x)
        del x
    del data

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                           scale_padding=0.2, returnfig=True, savefig=address, tight_layout=True,figsize =(3,2))
        plt.close("all")
        del df
        del date_list
        del rrng
        del i
        del right_now
        gc.collect()



        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 170 == 0:
          print(" ")

    print("")
    del dfs
    del suggs
    del index
    del all_n
    gc.collect()



def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [60]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd"]
download_data(symbols,"max","1wk")

Symbols : 1500
Data Folder Removed
 


In [128]:
how_many_future_candles = 5
how_many_past_candles = 5
each_row_past = 90

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 3
Processing File:
File Number 1: 10/151 20/151 30/151 40/151 50/151 60/151 70/151 80/151 90/151 100/151 110/151 120/151 130/151 140/151 150/151 
File Number 2: 10/151 20/151 30/151 40/151 50/151 60/151 70/151 80/151 90/151 100/151 110/151 120/151 130/151 140/151 150/151 
File Number 3: 10/315 20/315 30/315 40/315 50/315 60/315 70/315 80/315 90/315 100/315 110/315 120/315 130/315 140/315 150/315 160/315 170/315  
180/315 190/315 200/315 210/315 220/315 230/315 240/315 250/315 260/315 270/315 280/315 290/315 300/315 310/315 
 


617

In [129]:
#folder_name = '083812'
label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name,0)
    dim = (128, 128)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255.0
data.shape

(617, 128, 128)

In [130]:
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.1,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

xTrain : 555 \ xTest : 62
yn: 357 nn: 260


In [144]:
model = Sequential()

model.add(Conv2D(50,  (9, 9),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],1)))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(150,  (9, 9),activation="relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_105 (Conv2D)         (None, 120, 120, 50)      4100      
                                                                 
 max_pooling2d_96 (MaxPoolin  (None, 40, 40, 50)       0         
 g2D)                                                            
                                                                 
 conv2d_106 (Conv2D)         (None, 32, 32, 150)       607650    
                                                                 
 max_pooling2d_97 (MaxPoolin  (None, 10, 10, 150)      0         
 g2D)                                                            
                                                                 
 flatten_33 (Flatten)        (None, 15000)             0         
                                                                 
 dense_151 (Dense)           (None, 1024)            

In [ ]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(xTrain,yTrain,batch_size=8,epochs=25,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)


Epoch 1/25
70/70 [==============================] - 3s 29ms/step - loss: 0.7650 - accuracy: 0.5676 - val_loss: 0.6837 - val_accuracy: 0.5806
Epoch 2/25
70/70 [==============================] - 1s 16ms/step - loss: 0.6880 - accuracy: 0.5784 - val_loss: 0.6836 - val_accuracy: 0.5806
Epoch 3/25
70/70 [==============================] - 1s 16ms/step - loss: 0.6834 - accuracy: 0.5784 - val_loss: 0.6796 - val_accuracy: 0.5806
Epoch 4/25
70/70 [==============================] - 1s 16ms/step - loss: 0.6807 - accuracy: 0.5784 - val_loss: 0.6797 - val_accuracy: 0.5806
Epoch 5/25
70/70 [==============================] - 1s 16ms/step - loss: 0.6777 - accuracy: 0.5820 - val_loss: 0.6833 - val_accuracy: 0.5806
Epoch 6/25
70/70 [==============================] - 1s 16ms/step - loss: 0.6824 - accuracy: 0.5784 - val_loss: 0.6805 - val_accuracy: 0.5806
Epoch 7/25
70/70 [==============================] - 1s 16ms/step - loss: 0.6822 - accuracy: 0.5784 - val_loss: 0.6801 - val_accuracy: 0.5806
Epoch 8/25
70